In [1]:
import os
os.environ["MKL_NUM_THREADS"] = "4" # export MKL_NUM_THREADS=6
import numpy as np
import logging,time
#sys.path.append('../')
os.chdir('../')
from src.costs.full_shape_gradient import Full_shape_gradient

In [2]:
full_grad=Full_shape_gradient(path_config_file='config_file/config_full.ini')
output_folder_name='tmp/'+full_grad.config['other']['path_output']
os.mkdir(output_folder_name)

In [3]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
filehandler = logging.FileHandler('{}/log.txt'.format(output_folder_name), 'a')
formatter = logging.Formatter('%(levelname)s::%(message)s')
filehandler.setFormatter(formatter)
logger.addHandler(filehandler)

In [4]:
eps=1e-9
init_param=full_grad.init_param
len_param=len(init_param)
perturb=(2*np.random.random(len_param)-1)
new_param=init_param+eps*perturb
t0=time.time()
old_cost=full_grad.cost(init_param)
new_cost=full_grad.cost(new_param)
gradient=full_grad.shape_grad(init_param)
print('time used : {}s'.format(time.time()-t0))

INFO:root:Chi_B : 4.801965e-03, Chi_j = 1.432846e+14, EM cost 4.062312e-02
INFO:root:min_distance 1.919394e-01 m, Distance cost : 6.187334e-03
INFO:root:perimeter :5.567741e+01 m^2, perimeter cost : 0.000000e+00
INFO:root:maximal curvature 1.185919e+01 m^-1, curvature cost : 0.000000e+00
INFO:root:Total cost : 4.681045e-02
INFO:root:Chi_B : 4.801965e-03, Chi_j = 1.432846e+14, EM cost 4.062312e-02
INFO:root:min_distance 1.919394e-01 m, Distance cost : 6.187327e-03
INFO:root:perimeter :5.567741e+01 m^2, perimeter cost : 0.000000e+00
INFO:root:maximal curvature 1.185919e+01 m^-1, curvature cost : 0.000000e+00
INFO:root:Total cost : 4.681044e-02


time used : 14.768939018249512s


In [5]:
print('numerical cost obtained: {:10e} and {:10e}'.format(old_cost,new_cost))
print('numerical gradient : {:10e}'.format((new_cost-old_cost)/eps))
print('gradient computed : {:10e}'.format(np.dot(gradient,perturb)))

numerical cost obtained: 4.681045e-02 and 4.681044e-02
numerical gradient : -7.313152e+00
gradient computed : -7.313161e+00


In [6]:
import pickle
freq_save=100
def f(X, info):
    res = full_grad.cost(X)
    # display information
    if info['Nfeval']%freq_save == 0:
        print('Neval : {0:4d} \n saving the intermediate shape'.format(info['Nfeval']))
        with open('{}/intermediate{:}.res'.format(output_folder_name,info['Nfeval']),'wb') as output_file:
            pickle.dump(res,output_file)
    info['Nfeval'] += 1
    return res
def gradf(X, info):
    return full_grad.shape_grad(X)

In [ ]:
#We write the config before starting
with open('{}/config.ini'.format(output_folder_name), 'w') as tmp_file:
    full_grad.config.write(tmp_file)
import scipy.optimize
optimize_shape=scipy.optimize.minimize(f, full_grad.init_param, jac=gradf,args=({'Nfeval':0},),options={'maxiter':2000,'return_all':True})

INFO:root:Chi_B : 4.801965e-03, Chi_j = 1.432846e+14, EM cost 4.062312e-02
INFO:root:min_distance 1.919394e-01 m, Distance cost : 6.187334e-03
INFO:root:perimeter :5.567741e+01 m^2, perimeter cost : 0.000000e+00
INFO:root:maximal curvature 1.185919e+01 m^-1, curvature cost : 0.000000e+00
INFO:root:Total cost : 4.681045e-02


Neval :    0 
 saving the intermediate shape


INFO:root:Chi_B : 5.845777e-02, Chi_j = 2.802672e+14, EM cost 1.285246e-01
INFO:root:min_distance 7.929137e-03 m, Distance cost :   inf
INFO:root:perimeter :2.203626e+02 m^2, perimeter cost :   inf
INFO:root:maximal curvature 6.111695e+03 m^-1, curvature cost :   inf
INFO:root:Total cost :   inf
/users/home/robin/stellacage_code/src/costs/distance_shape_gradient.py:57: RuntimeWarning: invalid value encountered in multiply
  X=-1*vgradf[:,:,np.newaxis]*grad_d_min
/users/home/robin/stellacage_code/src/costs/perimeter_shape_gradient.py:26: RuntimeWarning: invalid value encountered in multiply
  return grad_f_e(self.c0,self.c1,perim)*grad_perim
INFO:root:Chi_B : 4.801965e-03, Chi_j = 1.432846e+14, EM cost 4.062312e-02
INFO:root:min_distance 1.919394e-01 m, Distance cost : 6.187334e-03
INFO:root:perimeter :5.567741e+01 m^2, perimeter cost : 0.000000e+00
INFO:root:maximal curvature 1.185919e+01 m^-1, curvature cost : 0.000000e+00
INFO:root:Total cost : 4.681045e-02
INFO:root:Chi_B : 5.845777

Neval :  100 
 saving the intermediate shape


INFO:root:Chi_B : 3.896895e-03, Chi_j = 1.246718e+14, EM cost 3.506484e-02
INFO:root:min_distance 1.984190e-01 m, Distance cost : 1.438815e-04
INFO:root:perimeter :5.581403e+01 m^2, perimeter cost : 0.000000e+00
INFO:root:maximal curvature 1.303110e+01 m^-1, curvature cost : 1.747054e-05
INFO:root:Total cost : 3.522619e-02
INFO:root:Chi_B : 3.853051e-03, Chi_j = 1.239839e+14, EM cost 3.484902e-02
INFO:root:min_distance 1.985566e-01 m, Distance cost : 1.715292e-04
INFO:root:perimeter :5.582276e+01 m^2, perimeter cost : 0.000000e+00
INFO:root:maximal curvature 1.405924e+01 m^-1, curvature cost : 4.420210e-02
INFO:root:Total cost : 7.922264e-02
INFO:root:Chi_B : 3.889957e-03, Chi_j = 1.245633e+14, EM cost 3.503078e-02
INFO:root:min_distance 1.984405e-01 m, Distance cost : 1.431847e-04
INFO:root:perimeter :5.581508e+01 m^2, perimeter cost : 0.000000e+00
INFO:root:maximal curvature 1.302526e+01 m^-1, curvature cost : 1.155223e-05
INFO:root:Total cost : 3.518552e-02
INFO:root:Chi_B : 3.86550

In [7]:
optimize_shape2=scipy.optimize.minimize(f, optimize_shape.x, method='CG', jac=gradf,options={'maxiter':100,'return_all':True})

temporary costs : 
 Regcoil 9.020159e-01 
 distance 1.940695e-03 
 perimeter 0.000000e+00
temporary costs : 
 Regcoil 2.385042e+00 
 distance   inf 
 perimeter   inf
main/full_gradient.py:56: RuntimeWarning: invalid value encountered in multiply
  X=-1*vgradf[:,:,np.newaxis]*grad_d_min
main/full_gradient.py:108: RuntimeWarning: invalid value encountered in multiply
  return grad_f_e(perim)*grad_perim
temporary costs : 
 Regcoil 9.020159e-01 
 distance 1.940695e-03 
 perimeter 0.000000e+00
temporary costs : 
 Regcoil 2.385042e+00 
 distance   inf 
 perimeter   inf
temporary costs : 
 Regcoil 1.821973e+00 
 distance   inf 
 perimeter   inf
temporary costs : 
 Regcoil 1.394647e+00 
 distance   inf 
 perimeter   inf
temporary costs : 
 Regcoil 9.103222e-01 
 distance   inf 
 perimeter   inf
temporary costs : 
 Regcoil 8.602175e-01 
 distance   inf 
 perimeter   inf
temporary costs : 
 Regcoil 8.260542e-01 
 distance   inf 
 perimeter 1.762486e+02
temporary costs : 
 Regcoil 8.561876e-01 
 

In [8]:
optimize_shape2

, -4.44209242e-03, -9.09378568e-03,
        4.43132379e-03,  2.92608384e-03, -3.93250179e-03,  3.00321819e-03,
        1.27330374e-02,  4.65397360e-03,  5.02558028e-03,  6.49201786e-03,
        1.13595748e-02,  8.09958102e-03, -1.05280464e-02, -2.76474076e-02,
        5.43312552e-04,  1.27070622e-03, -9.19776287e-04, -1.24948111e-03,
        1.48277873e-03, -5.07754542e-03,  1.11392206e-03, -3.01654234e-03,
        2.87208895e-03, -1.77054008e-03, -1.44284839e-04,  4.55909777e-04,
        2.32792762e-03, -7.84282926e-03, -4.68684678e-03, -2.09822842e-03,
       -2.79596395e-02, -4.63082942e-03,  1.42103728e-02,  3.11089992e-02,
        1.02343305e-02,  1.42786817e-03, -1.00550929e-02, -2.42364939e-03,
       -3.82764567e-03,  3.00158705e-03,  4.85544530e-03, -3.32408026e-03,
        2.42181863e-03, -3.40884748e-03, -1.29433343e-03, -6.83156362e-04,
        4.26554236e-03,  1.04360798e-02,  3.45197428e-04,  9.40787263e-04,
        1.35307901e-02,  1.39175725e-02, -9.57492879e-03, -1.532

In [4]:
%lprun -u 1 -f full_grad.grad_cost full_grad.grad_cost(full_grad.S_parametrization)

Timer unit: 1 s

Total time: 20.9241 s
File: main/full_gradient.py
Function: grad_cost at line 84

Line #      Hits         Time  Per Hit   % Time  Line Contents
    84                                               def grad_cost(self,paramS):
    85         1          0.2      0.2      1.1          S=Toroidal_surface(paramS,(self.ntheta_coil,self.nzeta_coil),self.Np)
    86         1         17.3     17.3     82.7          result=self.compute_gradient_of(S=S)
    87         1          0.0      0.0      0.0          I_vector,I_matrix=result['I1']
    88         1          0.6      0.6      2.7          X,Y=self.gradient_cost_distance(result['T'],S)
    89         1          2.8      2.8     13.1          theta,dtildetheta,dtheta,dSdtheta=S.get_theta_pertubation()
    90         1          0.1      0.1      0.3          grad=np.einsum('ija,oija,ij->o',I_vector,theta,S.dS/S.npts)+np.einsum('ijab,oijab,ij->o',I_matrix,dtildetheta,S.dS/S.npts)
    91         1          0.0      0.0      0.1

In [ ]:
print(optimize_shape)